## Load the Data

We will first download the DBPedia data and load into a Pandas dataframe.
We drop the `l2` and `l3` columns because we are only interested on `l1` column, to use the first level label (that contains 9 classes).


In [1]:
!curl -L -o dbpedia-classes.zip https://www.kaggle.com/api/v1/datasets/download/danofer/dbpedia-classes

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  165M  100  165M    0     0  76.4M      0  0:00:02  0:00:02 --:--:--  112M


In [2]:
!unzip dbpedia-classes.zip

Archive:  dbpedia-classes.zip
  inflating: DBPEDIA_test.csv        
  inflating: DBPEDIA_train.csv       
  inflating: DBPEDIA_val.csv         
  inflating: DBP_wiki_data.csv       


In [3]:
import pandas as pd

In [4]:
train_df = pd.read_csv("/content/DBPEDIA_train.csv")
test_df = pd.read_csv("/content/DBPEDIA_test.csv")

In [5]:
train_df.drop(columns=["l1", "l2"])
test_df.drop(columns=["l1", "l2"])

,text,l3
0,Liu Chao-shiuan (Chinese: 劉兆玄; pinyin: Liú Zhà...,PrimeMinister
1,"Michelle Maylene (born January 20, 1987) is an...",AdultActor
2,Hirfanlı Dam is a dam in Turkey. The developme...,Dam
3,Grote Prijs Stad Zottegem is a single-day road...,CyclingRace
4,"Johannes Petrus \""Hans\"" Nijman (September 23,...",MartialArtist
...,...,...
60789,Darwin Smith was CEO of Kimberly-Clark from 19...,BusinessPerson
60790,The HKRU Premiership is the main domestic comp...,SoccerLeague
60791,"Leo Harvey Diegel (April 20, 1899 – May 5, 195...",GolfPlayer
60792,"Clement Reginald \""Clem\"" Splatt (17 September...",AustralianRulesFootballPlayer


In [6]:
train_df = train_df.rename(columns={"text": "text", "l3": "label"})
test_df = test_df.rename(columns={"text": "text", "l3": "label"})

# Classic Markov

In [7]:
import re
from collections import defaultdict, Counter
import numpy as np
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return word_tokenize(text)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [8]:
class MarkovModel:
    def __init__(self):
        self.transitions = defaultdict(Counter)
        self.vocab = set()

    def train(self, texts):
        for text in texts:
            tokens = preprocess(text)
            tokens = ["<START>"] + tokens + ["<END>"]
            self.vocab.update(tokens)
            for i in range(len(tokens) - 1):
                self.transitions[tokens[i]][tokens[i + 1]] += 1

        self.vocab = list(self.vocab)
        self._calculate_probabilities()

    def _calculate_probabilities(self):
        self.probabilities = {}
        for prev_word, next_words in self.transitions.items():
            total = sum(next_words.values())
            self.probabilities[prev_word] = {
                word: count / total for word, count in next_words.items()
            }

    def sequence_log_prob(self, text):
        tokens = preprocess(text)
        tokens = ["<START>"] + tokens + ["<END>"]
        log_prob = 0.0
        for i in range(len(tokens) - 1):
            prev_word = tokens[i]
            next_word = tokens[i + 1]
            prob = self.probabilities.get(prev_word, {}).get(next_word, 1e-6)  # Laplace smoothing
            log_prob += np.log(prob)
        return log_prob


In [9]:
models = {}
classes = train_df['label'].unique()

for cls in classes:
    model = MarkovModel()
    model.train(train_df[train_df['label'] == cls]['text'])
    models[cls] = model


In [10]:
def classify(text, models):
    class_probs = {}
    for cls, model in models.items():
        class_probs[cls] = model.sequence_log_prob(text)
    return max(class_probs, key=class_probs.get)


In [11]:
predictions = [classify(text, models) for text in test_df['text']]
accuracy = np.mean(np.array(predictions) == test_df['label'].values)
print(f"Acurácia (Markov Clássico): {accuracy:.4f}")

Acurácia (Markov Clássico): 0.8760


# Métricas de avaliação do modelo

In [12]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from matplotlib import pyplot as plt

# gerar matriz de confusao
cm = confusion_matrix(test_df['label'], predictions, labels=classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)

# plotar
fig, ax = plt.subplots(figsize=(96, 96))
disp.plot(xticks_rotation="vertical", cmap="Oranges", values_format='d', ax=ax)

plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.title("Matriz de Confusão - Markov", fontsize=18)
plt.tight_layout()
plt.show()

In [13]:
from sklearn.metrics import classification_report

print(classification_report(test_df['label'], predictions, target_names=classes))

                                   precision    recall  f1-score   support

                          Senator       0.93      0.94      0.93       485
                            Album       0.90      0.74      0.81       211
                         Mountain       0.98      0.97      0.98       481
                  AcademicJournal       0.95      0.95      0.95       483
                 HistoricBuilding       0.76      0.72      0.74       480
                          Reptile       0.98      0.83      0.90        72
                     MilitaryUnit       0.84      0.53      0.65       102
                            Judge       0.93      0.93      0.93       483
                      ChessPlayer       0.98      0.91      0.94       481
                       TradeUnion       0.96      0.62      0.75        37
                          Musical       0.87      0.55      0.67       218
                           Insect       0.98      0.94      0.96       399
                        